# Loading our data
We will be working with "institutional strength" stock level data through this module. The stock universe is known as the All Countries World Index (ACWI). These stocks are equities drawn from developed (DM) and emerging markets (EM) and represent around 6000 investable equities with data drawn from the year 2000 through to the end of 2020. This is considered to be a very broad universe of stocks and encompasses the full range of sectors and companies from around 50 countries. We will be using a number of critical features associated with each stock which are drawn from exchange level data, such as pricing, fundamentals, from financial reporting, and sentiment scoring drawn from new-flow. 

PLEASE NOTE THAT THE EQUITIES HAVE BEEN ANONYMISED AND NOTE THE LICENSING OF THIS DATASET PROHIBITS DISTIBUTION OR USE OF THIS DATA BEYOND THIS COURSE.

This dataset will be used to take you through the process of loading, pre-processing and then using various learners and techniques to build investment models, to allow us to select the "best" stocks to generate stronger return characteristics than simply investing in the index would give us.

The file we will be working with is named ```ACWI fundamental data.xlsx```. If one was to load this file in excel then one would find financial data on around 6000 companies spread across a number of different worksheets. 
NB: Files in the XLSX format can be very slow to work with in python so it is normally a good idea to convert the worksheets to separate CSV files. In this notebook, we will see how this can be done automatically.

# Pandas ... lovely and yet horrid
We will be using Pandas for ease in this module which makes operations on smaller data-sets easy and generic. However, before you engineer your own investment pipeline with Pandas, bear in mind that in the real world it is not advised. Remember, anything that looks like SQL should live on a server, not on a client machine. Better to use an enterprise strength SQL back end to do enterprise back end stuff.


In [1]:
# package for working with tabular data
import pandas as pd 

# package for timing runtime
import time

# package for navigating the operating system
import os       

Let us start by loading ```ACWI fundamental data.xlsx```.

In [2]:
path = "ACWI fundamental data.xlsx"
start_time = time.time()
xls = pd.ExcelFile(path)
print("It took %s seconds to load the .xlsx file." % (time.time() - start_time))

FileNotFoundError: [Errno 2] No such file or directory: 'ACWI fundamental data.xlsx'

On my machine, it took 635 seconds to load the data. Let us look at the worksheet names in the XLSX file. Let us look at the worksheets in ```xls```. 

In [6]:
for i, sheet_name in enumerate(xls.sheet_names):
    print(i, sheet_name)

0 TotalReturn_BaseCCY
1 EPS
2 GroupRank_NPL_Loans_egSector
3 Mkt_Beta_2yrs
4 NI_Sales
5 Diag SentimentNews
6 Diag EPSGrowth3yrAverage_Rothko
7 OperationalLev_USD
8 Rev_Quality
9 Diag FreeCashFlowpershare
10 EBITDA_SALES
11 Diag TotalYield
12 Diag DebtToEquity
13 Diag PE
14 Diag EPS
15 Beta_DnSide_24m
16 GroupRank_Diag FreeCashFlowpers
17 Diag ROE


There are a total of 18 worksheets. We will convert these worksheets into separate CSV files, and store them in a folder named ```Data```.
Please see the data_dictionary.md to understand more about the features. 

In [7]:
dir_name = "Data" # name of the folder that we store the CSV files in

try:
    os.mkdir("Data") # make a folder named Data
except:
    pass # pass if the folder already exists

for sheet_name in xls.sheet_names:
    path = "Data/" + sheet_name + ".csv" # path to where we want to save the CSV file
    print("Saving " + path + " ...", end='')
    
    # load the worksheet into a dataframe
    df = pd.read_excel(
        xls, 
        sheet_name, 
        skiprows = 8 # the data does not start until after row 8
    )
    
    # convert the date column names into a consistent format
    dates = df.columns[4:]
    dates = pd.to_datetime(dates)
    dates = list(dates.strftime('%Y-%m-%d')) # convert the dates to the form YYYY-MM-DD
    df.columns = list(df.columns[:4]) + dates
    
    # save the worksheet as a CSV file in the data folder
    df.to_csv(
        path, 
        index=False # prevent the index column being saved
    )
    print(" DONE")

Saving Data/TotalReturn_BaseCCY.csv ... DONE
Saving Data/EPS.csv ... DONE
Saving Data/GroupRank_NPL_Loans_egSector.csv ... DONE
Saving Data/Mkt_Beta_2yrs.csv ... DONE
Saving Data/NI_Sales.csv ... DONE
Saving Data/Diag SentimentNews.csv ... DONE
Saving Data/Diag EPSGrowth3yrAverage_Rothko.csv ... DONE
Saving Data/OperationalLev_USD.csv ... DONE
Saving Data/Rev_Quality.csv ... DONE
Saving Data/Diag FreeCashFlowpershare.csv ... DONE
Saving Data/EBITDA_SALES.csv ... DONE
Saving Data/Diag TotalYield.csv ... DONE
Saving Data/Diag DebtToEquity.csv ... DONE
Saving Data/Diag PE.csv ... DONE
Saving Data/Diag EPS.csv ... DONE
Saving Data/Beta_DnSide_24m.csv ... DONE
Saving Data/GroupRank_Diag FreeCashFlowpers.csv ... DONE
Saving Data/Diag ROE.csv ... DONE


We can use the ```os.listdir()``` function to list the files in the ```Data``` folder.

In [8]:
os.listdir("Data")

['Beta_DnSide_24m.csv',
 'Diag DebtToEquity.csv',
 'Diag EPS.csv',
 'Diag EPSGrowth3yrAverage_Rothko.csv',
 'Diag FreeCashFlowpershare.csv',
 'Diag PE.csv',
 'Diag ROE.csv',
 'Diag SentimentNews.csv',
 'Diag TotalYield.csv',
 'EBITDA_SALES.csv',
 'EPS.csv',
 'GroupRank_Diag FreeCashFlowpers.csv',
 'GroupRank_NPL_Loans_egSector.csv',
 'Mkt_Beta_2yrs.csv',
 'NI_Sales.csv',
 'OperationalLev_USD.csv',
 'Rev_Quality.csv',
 'TotalReturn_BaseCCY.csv']

Now let us see how fast it is to load these CSV files compared to the original XLSX file.

In [9]:
worksheets = os.listdir('Data')

start_time = time.time()
for worksheet in worksheets:
    print(worksheet + " loading ", end='')
    df = pd.read_csv("Data/" + worksheet)
    print("DONE")
print("It took %s seconds to load the .csv files." % (time.time() - start_time))

Beta_DnSide_24m.csv loading DONE
Diag DebtToEquity.csv loading DONE
Diag EPS.csv loading DONE
Diag EPSGrowth3yrAverage_Rothko.csv loading DONE
Diag FreeCashFlowpershare.csv loading DONE
Diag PE.csv loading DONE
Diag ROE.csv loading DONE
Diag SentimentNews.csv loading DONE
Diag TotalYield.csv loading DONE
EBITDA_SALES.csv loading DONE
EPS.csv loading DONE
GroupRank_Diag FreeCashFlowpers.csv loading DONE
GroupRank_NPL_Loans_egSector.csv loading DONE
Mkt_Beta_2yrs.csv loading DONE
NI_Sales.csv loading DONE
OperationalLev_USD.csv loading DONE
Rev_Quality.csv loading DONE
TotalReturn_BaseCCY.csv loading DONE
It took 9.068223237991333 seconds to load the .csv files.


It took 9 seconds to load all of the CSV files. Let us take a look at one of the CSV files.

In [10]:
df = pd.read_csv("Data/TotalReturn_BaseCCY.csv")
df.head()

,NAME,TICKER,SECTOR,COUNTRY,2020-09-20,2020-08-31,2020-07-31,2020-06-30,2020-05-31,2020-04-30,...,2000-10-31,2000-09-30,2000-08-31,2000-07-31,2000-06-30,2000-05-31,2000-04-30,2000-03-31,2000-02-29,2000-01-31
0,Aberdeen Asset Management PLC,1,Financials,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,...,0.045730,-0.120105,0.207501,-0.016259,0.201910,-0.061498,-0.226854,0.248221,0.435955,0.224755
1,Abbey National Plc,2,Financials,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,...,0.038953,0.095348,0.104281,-0.062044,-0.100452,0.161520,-0.130285,0.221048,-0.072973,-0.241147
2,Chubb Plc,3,Industrials,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001878,-0.129013,0.024722,-0.062093,0.026172,0.031064,0.119904,0.222242,0.026770,-0.111193
3,Kidde Plc,4,Consumer Discretionary,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Amec Foster Wheeler plc,5,Industrials,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000716,0.034200,0.223598,0.179246,-0.150297,0.251863,-0.096557,0.234714,-0.331340,-0.077761


This CSV contains the monthly return data of around 6000 companies from 2000 to 2020.